
# Python Lesson: Static Methods, Enums, and Decorators

This notebook covers three advanced Python concepts:
1. **Static Methods**: Utility methods associated with a class but independent of instance or class data.
2. **Enums**: A way to define a fixed set of constants.
3. **Decorators**: Functions that enhance or modify other functions' behavior dynamically.

We'll also explore practical use cases and combine these concepts into a real-world example: **Role-Based Access Control**.
    


## Static Methods

### What Are Static Methods?

A **static method** belongs to a class but:
- Does not have access to the instance (`self`) or class (`cls`).
- Is defined with the `@staticmethod` decorator.

### Use Case
Static methods are useful for utility functions that logically belong to a class but don’t require instance or class context.

### Example: A Utility for Validating Data
    

In [ ]:

class Validator:
    @staticmethod
    def is_positive(number):
        """Check if a number is positive"""
        return number > 0

# Usage
print(Validator.is_positive(10))  # True
print(Validator.is_positive(-5))  # False
    


## Enums

### What Are Enums?

**Enums** (Enumerations) are a way to define a set of named constants. They:
- Improve code readability.
- Help avoid "magic strings" or numbers.

### Use Case
Enums are ideal for defining categories like user roles, states, or directions.

### Example: Using Enums for User Roles
    

In [ ]:

from enum import Enum

class Role(Enum):
    ADMIN = "admin"
    USER = "user"
    GUEST = "guest"

# Usage
print(Role.ADMIN)          # Role.ADMIN
print(Role.ADMIN.value)    # "admin"

# Compare enums
if Role.ADMIN == Role.ADMIN:
    print("Both are admins")
    


## Combining Static Methods and Enums

Static methods can work with enums to define utility logic for validating or managing enums.

### Example: Checking Permissions with Static Methods and Enums
    

In [ ]:

class RoleManager:
    @staticmethod
    def has_permission(role):
        """Check if a role has admin-level permissions"""
        return role == Role.ADMIN

# Usage
print(RoleManager.has_permission(Role.ADMIN))  # True
print(RoleManager.has_permission(Role.USER))   # False
    


## Decorators

### What Are Decorators?

A **decorator** is a function that:
1. Takes another function as input.
2. Enhances or modifies its behavior.
3. Returns the enhanced function.

### Use Case
Decorators are useful for:
- Logging
- Validating input/output
- Restricting access (e.g., role-based access control)

### Example: A Simple Logging Decorator
    

In [4]:
from functools import wraps
from typing import Callable 

# basic decorator
def log_execution(func: Callable): # receives the function as argument
    """Decorator to log function calls"""
    def wrapper(*args, **kwargs):  # the function tha executes the received function `func`
        display(f"Executing {func.__name__} with args={args} kwargs={kwargs}")
        result = func(*args, **kwargs)
        display(f"Finished {func.__name__} with result={result}")
        return result
    return wrapper

# advanced decorator
def log_execute_with_prefix(prefix: str): # receives the argument
    def decorator(func: Callable):# receives the function as argument
        @wraps(func) # tells the wrapper function where to inject the *args, **kwargs  from in our case it takes it from the above func argument
        def wrapper(*args, **kwargs):  # the function tha executes the received function `func`
            display(f"{prefix}: Executing {func.__name__} with args={args} kwargs={kwargs}")
            result = func(*args, **kwargs)
            display(f"{prefix}: Finished {func.__name__} with result={result}")
            return result
        return wrapper
    return decorator

@log_execution
def add(a, b):
    return a + b


@log_execute_with_prefix("fancy")
def add_with_prefix(a, b):
    return a + b
# Usage
# add(5, 7)


""" 
@log_execute_with_prefix("fancy") => log_execute_with_prefix(prefix="fancy") and inject add_with_prefix(5, 7) to decorator = decorator(add_with_prefix(5, 7))

"""

add_with_prefix(5, 7)

'fancy: Executing add_with_prefix with args=(5, 7) kwargs={}'

'fancy: Finished add_with_prefix with result=12'

12


## Advanced Decorator Examples

### Why Use Decorators?

Decorators abstract repetitive tasks, dynamically enhance functions, and keep your code clean. Here are some advanced examples:
1. **Timing Decorator**: Measure execution time.
2. **Caching Decorator**: Cache expensive computations.
3. **Retry Decorator**: Retry a function on failure.
4. **Authorization Decorator**: Restrict access based on roles.
5. **Validation Decorator**: Validate function parameters.
6. **Debugging Decorator**: Log function calls and results.
    

In [7]:

import time
from functools import wraps
from typing import Callable

# basic
def timing_decorator(func): # receives the function as argument
    """Decorator to measure the execution time of a function"""
    def wrapper(*args, **kwargs): # the function tha executes the received function `func`
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Execution time for {func.__name__}: {end_time - start_time:.4f} seconds")
        return result
    return wrapper

# advanced
def timing_decorator_with_unit(unit="seconds"): # receives the argument
    def decorator(func: Callable): # receives the function as argument
        @wraps(func) # tells the wrapper function where to inject the *args, **kwargs  from in our case it takes it from the above func argument
        def wrapper(*args, **kwargs): # the function tha executes the received function `func`
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            if unit == "milliseconds":
                # elapsed_time = elapsed_time * 1000
                elapsed_time *= 1000
                print(f"Execution time for {func.__name__}: {elapsed_time:.4f} ms")
            else:
                print(f"Execution time for {func.__name__}: {elapsed_time:.4f} seconds")
            
            return result
        return wrapper
    return decorator


@timing_decorator
def slow_function():
    """Simulates a slow function by sleeping for 2 seconds"""
    time.sleep(2)
    return "Finished slow function!"


@timing_decorator_with_unit(unit="milliseconds")
def slow_function_with_units():
    """Simulates a slow function by sleeping for 2 seconds"""
    time.sleep(2)
    return "Finished slow function!"

# Usage
slow_function_with_units()
    

Execution time for slow_function_with_units: 2000.1242 ms


'Finished slow function!'

In [8]:

import time
from functools import wraps
from typing import Callable 

cache = {}
# basic
def cache_decorator(func): # receives the function as argument
    """Decorator to cache the results of function calls""" 
    def wrapper(*args): # the function tha executes the received function `func`
        if args in cache:
            print(f"Cache hit for {args}")
            return cache[args]
        print(f"Cache miss for {args}")
        result = func(*args)
        cache[args] = result
        return result
    return wrapper

# advanced
def cache_decorator_with_limit(limit=None): # receives the argument
    def decorator(func: Callable): # receives the function as argument
        @wraps(func) # tells the wrapper function where to inject the *args, **kwargs  from in our case it takes it from the above func argument
        def wrapper(*args, **kwargs): # the function tha executes the received function `func`
            if args in cache:
                print(f"Cache hit for {args}")
                return cache[args]
            
            print(f"Cache miss for {args}")
           
            result = func(*args)
            cache[args] = result
            
            if limit and len(cache) > limit:
                """ 
                
                next: goes to the keys of the dict one by one
                iter: loops over list
                {
                    "1": "Afdasdfa",
                    "2": "afdasdfA",
                    "3": "afsdasfdas
                }
                keys = ["1", "2", "3"]
                
                next(keys) = "1" , ["2", "3"]
                next(keys) = "2", ["3"]
                next(keys) = "3" 
                
                last_key = next(iter(keys)) = "3"
                    first loop
                        next(keys) = "1" , ["2", "3"]
                        last_key = "1"
                    second loop
                        next(keys) = "2", ["3"]
                        last_key = "2"
                    third loop
                        next(keys) = "3" 
                        last_key = "3"
                    ፨  last_key = "3"
                
                """
                oldest_key = next(iter(cache))
                print(f"Cache limit exceeded. Removing oldest entry: {oldest_key}")
            
                del cache[oldest_key]
            
            return result
            
        return wrapper
    return decorator

@cache_decorator
def add(a, b):
    """Add two numbers"""
    return a + b

@cache_decorator_with_limit(limit=2)
def add_with_limit_cache(a, b):
    """Add two numbers"""
    return a + b


# # Usage
# print(add(2, 3))  # Cache miss, result is calculated
# print(add(2, 3))  # Cache hit, result is retrieved from cache
# print(add(5, 7))  # Cache miss


# Usage
print(add_with_limit_cache(2, 3))  # Cache miss
print(add_with_limit_cache(2, 3))  # Cache hit
print(add_with_limit_cache(5, 7))  # Cache miss
print(add_with_limit_cache(8, 1))  # Cache miss, and the oldest cache (2, 3) will be removed
    

Cache miss for (2, 3)
5
Cache hit for (2, 3)
5
Cache miss for (5, 7)
12
Cache miss for (8, 1)
Cache limit exceeded. Removing oldest entry: (2, 3)
9



## Summary

### What You Learned
1. **Static Methods**:
   - Use for utility functions not dependent on instance or class data.

2. **Enums**:
   - Use to define a fixed set of related constants, improving readability.

3. **Decorators**:
   - Use to dynamically enhance or modify the behavior of functions.

### Advanced Topics Covered
- Timing, caching, retry, validation, and debugging decorators.
- Role-based access control combining enums, static methods, and decorators.

### Next Steps
- Experiment with creating your own decorators.
- Combine multiple decorators for advanced functionality.
    